In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from nlppreprocess import NLP

In [2]:
train_dataset=pd.read_csv("train.csv")

In [3]:
train_dataset.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [4]:
def dropping_columns(dataset,list_clolumns):
    dataset=dataset.drop(list_clolumns,axis=1)
    return dataset

In [5]:
train_dataset=dropping_columns(train_dataset,["id","keyword","location"])

In [6]:
train_dataset.head()

,text,target
0,Our Deeds are the Reason of this #earthquake M...,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are ...,1
3,"13,000 people receive #wildfires evacuation or...",1
4,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
y=train_dataset.iloc[:,-1].values

In [8]:
y.shape=(-1,1)

In [9]:
train_dataset=dropping_columns(train_dataset,["target"])


In [10]:
train_dataset.tail()

,text
7608,Two giant cranes holding a bridge collapse int...
7609,@aria_ahrary @TheTawniest The out of control w...
7610,M1.94 [01:04 UTC]?5km S of Volcano Hawaii. htt...
7611,Police investigating after an e-bike collided ...
7612,The Latest: More Homes Razed by Northern Calif...


In [11]:
def cleaning_text(dataset):
    from nltk.stem.porter import PorterStemmer
    corpus=[]
    ps=PorterStemmer()
    for i in range(dataset.shape[0]):
        tweet=re.sub("[^a-zA-Z]"," ",dataset["text"][i])
        tweet=tweet.lower()
        tweet=tweet.split()
        tweet=[word for word in tweet if not word in set(stopwords.words("english"))]
        tweet=[ ps.stem(word) for word in tweet if not word in set(stopwords.words("english"))]
        tweet=" ".join(tweet)
        corpus.append(tweet)
    return corpus

In [12]:
nlp=NLP()

In [13]:
train_dataset["text"]=train_dataset["text"].apply(nlp.process)

In [14]:
train_dataset.tail()

,text
7608,Two giant cranes holding bridge collapse into ...
7609,aria ahrary TheTawniest out control wild fir...
7610,M UTC km S Volcano Hawaii http t co zD...
7611,Police investigating after e bike collided wit...
7612,Latest More Homes Razed by Northern Californi...


In [15]:
train_dataset["text"]=train_dataset["text"].apply(lambda x:x.lower())

In [16]:
vocab=29
words=15000
embedding=256

In [17]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer=Tokenizer(num_words=words,oov_token="<oov>")
tokenizer.fit_on_texts(train_dataset["text"])

In [18]:
sequences=tokenizer.texts_to_sequences(train_dataset["text"])

In [19]:
sequences

[[88, 4410, 16, 813, 223, 110, 1541, 4411, 62],
 [161, 31, 190, 690, 6565, 6566, 1151],
 [1673,
  1542,
  1857,
  5,
  643,
  16,
  108,
  6567,
  12,
  1674,
  27,
  224,
  1857,
  5,
  643,
  1330,
  16,
  1045],
 [42, 4412, 1423, 224, 1330, 5, 63],
 [21, 73, 1152, 292, 14, 6568, 1675, 238, 14, 1423, 6569, 48, 154],
 [2797, 350, 63, 1331, 814, 5, 6570, 529, 1091, 372, 31, 4413, 1423],
 [182, 51, 775, 249, 1153, 776, 225, 1676, 5, 6571, 984, 2347, 1424],
 [13, 19, 191, 1858, 6, 32, 80, 31, 5, 4414],
 [47, 7, 52, 224, 1250, 36, 5, 554, 815, 691],
 [13, 19, 2348, 440, 214, 88, 232],
 [530, 42, 597, 14, 239, 419, 598],
 [816,
  511,
  2798,
  192,
  2799,
  3432,
  644,
  599,
  169,
  5,
  511,
  2798,
  37,
  126,
  293,
  37,
  126,
  293,
  6572,
  225],
 [2800, 225, 1543, 6573, 2798, 555, 13, 120, 667, 3433],
 [182, 5, 4415, 849, 23, 1544, 4415],
 [193, 154, 392, 5, 2801, 99, 66, 304],
 [37, 7, 20, 70],
 [86, 4416],
 [294, 1677],
 [9, 99, 746],
 [37, 6574],
 [2802],
 [927, 512],
 [8

In [20]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
padded=pad_sequences(sequences,maxlen=vocab)

In [21]:
padded.shape

(7613, 29)

In [22]:
from tensorflow import keras
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import GlobalAveragePooling1D
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Bidirectional
from keras.layers import BatchNormalization
from keras.optimizers import RMSprop
from keras.layers import MaxPooling1D

Using TensorFlow backend.


In [23]:
test_data=pd.read_csv("test.csv")

In [24]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [25]:
test_id=test_data["id"]
test_data=dropping_columns(test_data,["id","location","keyword"])

In [26]:
test_data.head()

,text
0,Just happened a terrible car crash
1,"Heard about #earthquake is different cities, s..."
2,"there is a forest fire at spot pond, geese are..."
3,Apocalypse lighting. #Spokane #wildfires
4,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
test_data["text"]=test_data["text"].apply(nlp.process)

In [28]:
test_data["text"]=test_data["text"].apply(lambda x:x.lower())

In [29]:
test_sequences=tokenizer.texts_to_sequences(test_data["text"])
test_padded=pad_sequences(test_sequences,maxlen=vocab)

In [30]:
test_padded.shape

(3263, 29)

In [31]:
model=Sequential()
model.add(Embedding(words,embedding,input_length=vocab))
model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
#model.add(Bidirectional(LSTM(units=64,return_sequences=True)))
#model.add(GlobalAveragePooling1D())
#model.add(Bidirectional(LSTM(units=64)))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dense(1000,activation="relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.08))
model.add(Dense(1000,activation="relu"))
model.add(BatchNormalization())
#model.add(Dropout(0.05))
model.add(Dense(500,activation="relu"))
model.add(BatchNormalization())
model.add(Dense(1,activation="sigmoid"))
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=["accuracy"])

In [32]:
epoch=100
validation=0.1
batch=256

In [33]:
model.fit(padded,y,batch_size=batch,epochs=epoch,validation_split=validation,verbose=1)

C:\Users\akprp\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 6851 samples, validate on 762 samples
Epoch 1/100
6851/6851 [==============================] - 14s 2ms/step - loss: 0.7267 - accuracy: 0.6907 - val_loss: 0.6833 - val_accuracy: 0.5341
Epoch 2/100
6851/6851 [==============================] - 13s 2ms/step - loss: 0.2447 - accuracy: 0.9085 - val_loss: 0.6790 - val_accuracy: 0.5341
Epoch 3/100
6851/6851 [==============================] - 13s 2ms/step - loss: 0.1110 - accuracy: 0.9650 - val_loss: 0.6702 - val_accuracy: 0.5341
Epoch 4/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0708 - accuracy: 0.9752 - val_loss: 0.6498 - val_accuracy: 0.5433
Epoch 5/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0493 - accuracy: 0.9809 - val_loss: 0.6338 - val_accuracy: 0.5919
Epoch 6/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0431 - accuracy: 0.9847 - val_loss: 0.6390 - val_accuracy: 0.5945
Epoch 7/100
6851/6851 [==============================] - 12s 2ms/step - loss:

6851/6851 [==============================] - 13s 2ms/step - loss: 0.0191 - accuracy: 0.9891 - val_loss: 1.5030 - val_accuracy: 0.7283
Epoch 57/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0181 - accuracy: 0.9888 - val_loss: 1.5286 - val_accuracy: 0.7388
Epoch 58/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0169 - accuracy: 0.9902 - val_loss: 1.4553 - val_accuracy: 0.7336
Epoch 59/100
6851/6851 [==============================] - 11s 2ms/step - loss: 0.0171 - accuracy: 0.9898 - val_loss: 1.4291 - val_accuracy: 0.7349
Epoch 60/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0165 - accuracy: 0.9904 - val_loss: 1.4659 - val_accuracy: 0.7362
Epoch 61/100
6851/6851 [==============================] - 12s 2ms/step - loss: 0.0161 - accuracy: 0.9914 - val_loss: 1.4808 - val_accuracy: 0.7415
Epoch 62/100
6851/6851 [==============================] - 13s 2ms/step - loss: 0.0162 - accuracy: 0.9907 - val_loss: 1.4617 - val_a

In [38]:
predict=model.predict_classes(test_padded)

In [39]:
predict.shape=(3263)

In [36]:
predict

array([0, 0, 1, ..., 1, 1, 1])

In [40]:
pd.DataFrame({"id":test_id,"target":predict}).to_csv("sample_submission.csv",index=False,header=True)